In [25]:
%%HTML
<style>
    body {
        --vscode-font-family: "ComicSansMS"
    }
</style>

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Testing out LDO_functions.py

In [27]:
## Calling the function from LDO_functions.py
from LDO_functions import estimate_amplifier_stages, estimate_GBW_stages
# Test the functions using available variables
csv_file = "sf_nmos_char_vg_1_3.csv"
PSRR_target_dB = -40
f_bw = 1e6
I_load = 10e-3
PM_target = 60
C_load = 11.4e-12
kgm_max = 25
I_budget = (5/100) * I_load  # 5% of I_load

In [28]:
GBW_stage1, GBW_stage2 = estimate_GBW_stages(PSRR_target_dB, f_bw, PM_target, C_load)

print(f"Estimated GBW_stage1: {GBW_stage1*1e-9:.2f} GHz")
print(f"Estimated GBW_stage2: {GBW_stage2*1e-9:.2f} GHz")

Estimated GBW_stage1: 0.92 GHz
Estimated GBW_stage2: 0.35 GHz


In [29]:
# Estimate amplifier stages using the current test parameters
result_test = estimate_amplifier_stages(csv_file, GBW_stage1, GBW_stage2, C_load, kgm_target=25)
print("Amplifier stages estimation (test):")
for k, v in result_test.items():
    if isinstance(v, float) or isinstance(v, np.floating):
        print(f"{k}: {v:.4e}")
    else:
        print(f"{k}: {v}")

Stage 2 required current (I_req_stage2): 0.9925 mA
Stage 2 input capacitance (C_in_stage2): 947.78 pF
Stage 1 required current (I_req_stage1): 220.0437 mA
Total required current (I_req_total): 221.0362 mA
Amplifier stages estimation (test):
I_req_stage2: 9.9251e-04
C_in_stage2: 9.4778e-10
I_req_stage1: 2.2004e-01
I_req_total: 2.2104e-01
details_stage2: {'idx_kgm': 9, 'kgm': np.float64(25.00386507127838), 'kcsg_n': np.float64(3.514438674718297e-07), 'kcgd_n': np.float64(3.017461437348191e-07), 'cgs': np.float64(3.4881190860395424e-10), 'cgd': np.float64(2.994863705182221e-10), 'C_in': np.float64(9.477846496403986e-10)}


In [30]:
def estimate_gbw_and_amplifier_stages(PSRR_target_dB, f_bw, PM_target, C_load, csv_file, kgm_target=25):
    """
    Combines GBW estimation and amplifier stage estimation.

    Returns:
        result: dict containing GBW_stage1, GBW_stage2, and amplifier stage estimation results.
    """
    GBW_stage1, GBW_stage2 = estimate_GBW_stages(PSRR_target_dB, f_bw, PM_target, C_load)
    amp_result = estimate_amplifier_stages(csv_file, GBW_stage1, GBW_stage2, C_load, kgm_target=kgm_target)
    result = {
        "GBW_stage1": GBW_stage1,
        "GBW_stage2": GBW_stage2,
        "amplifier_stages": amp_result
    }
    return result

In [31]:
## Calling the function from LDO_functions.py
from LDO_functions import estimate_amplifier_stages, estimate_GBW_stages, estimate_gbw_and_amplifier_stages
# Test the functions using available variables
csv_file = "sf_nmos_char_vg_1_3.csv"
PSRR_target_dB = -40
f_bw = 1e6
I_load = 10e-3
PM_target = 60
C_load = 11.4e-12
kgm_max = 25
I_budget = (5/100) * I_load  # 5% of I_load

In [32]:
# Test the combined GBW and amplifier stage estimation function
result_combined = estimate_gbw_and_amplifier_stages(
    PSRR_target_dB, f_bw, PM_target, C_load, csv_file, kgm_target=kgm_max
)
print("Combined GBW and amplifier stage estimation:")
for key, value in result_combined.items():
    print(f"{key}: {value}")

Stage 2 required current (I_req_stage2): 0.9925 mA
Stage 2 input capacitance (C_in_stage2): 947.78 pF
Stage 1 required current (I_req_stage1): 220.0437 mA
Total required current (I_req_total): 221.0362 mA
Combined GBW and amplifier stage estimation:
GBW_stage1: 923760430.7034009
GBW_stage2: 346410161.51377535
amplifier_stages: {'I_req_stage2': np.float64(0.0009925110121089637), 'C_in_stage2': np.float64(9.477846496403986e-10), 'I_req_stage1': np.float64(0.22004367295343424), 'I_req_total': np.float64(0.2210361839655432), 'details_stage2': {'idx_kgm': 9, 'kgm': np.float64(25.00386507127838), 'kcsg_n': np.float64(3.514438674718297e-07), 'kcgd_n': np.float64(3.017461437348191e-07), 'cgs': np.float64(3.4881190860395424e-10), 'cgd': np.float64(2.994863705182221e-10), 'C_in': np.float64(9.477846496403986e-10)}}


In [ ]:

PSRR_sweep = np.arange(-50, 1, 5)
I_budget = (5/100) * I_load  # 5% of I_load
I_total_list = []

for psrr in PSRR_sweep:
    result = estimate_gbw_and_amplifier_stages(
        psrr, f_bw, PM_target, C_load, csv_file, kgm_target=kgm_max
    )
    I_total = result['amplifier_stages']['I_req_total']
    I_total_list.append(I_total)


plt.figure(figsize=(7, 4))
plt.plot(PSRR_sweep, I_total_list, marker='o')
plt.hlines(I_budget, xmin=PSRR_sweep.min(), xmax=PSRR_sweep.max(), color='r', linestyle='--', label='I_budget (5% of I_load)')
plt.legend()
plt.xlabel('PSRR Target (dB)')
plt.yscale('log')
plt.ylabel('Total Required Current (mA)')
plt.title('I_total vs PSRR Target')
plt.grid(True)
plt.show()

# Find the PSRR value where I_total crosses I_budget
I_total_arr = np.array(I_total_list)
cross_idx = np.where(I_total_arr < I_budget)[0]
if len(cross_idx) > 0:
    idx = cross_idx[0]
    psrr_cross = PSRR_sweep[idx]
    I_cross = I_total_arr[idx]
    plt.scatter(psrr_cross, I_cross, color='red', zorder=5)
    plt.annotate(f'PSRR={psrr_cross} dB\nI_total={I_cross:.2e} A',
                 (psrr_cross, I_cross),
                 textcoords="offset points", xytext=(10,10), ha='left', color='red')